In [1]:
# Use low power tech to sleep until a rocker switch 
# flips and use its setting to send mqtt controls out


No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [8]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [145]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --quiet --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py


Sent 258 lines (8400 bytes) to lib/uasyncio/__init__.py.
Sent 315 lines (9509 bytes) to lib/uasyncio/core.py.
Sent 94 lines (2691 bytes) to lib/uasyncio/queues.py.
Sent 28 lines (846 bytes) to lib/uasyncio/synchro.py.
Sent 626 lines (22505 bytes) to mqtt_as.py.


In [22]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinyellow    4
pingreen     21
pinred       18
pinrocker    2
boardname    esp32flip
deepsleepsec 15


Sent 9 lines (176 bytes) to config.txt.


In [25]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinyellow    4
pingreen     21
pinred       18
pinrocker    2
boardname    esp32flip
deepsleepsec 600


Sent 9 lines (170 bytes) to config.txt.


In [10]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [11]:
%sendtofile main.py

import machine, time, esp32
rtc = machine.RTC()
def ftime():
    d = rtc.datetime()
    return time.mktime((d[0], d[1], d[2], d[4], d[5], d[6], 0, 0)) + (d[7]//1000)/1000

timestartup = ftime()
timeconnected = 0

resetcause = ["0", "PWRON", "HARD", "WDT", "DEEPSLEEP", "SOFT"][machine.reset_cause()]+"_RESET"
wakereason = ["0", "1", "EXT0", "EXT1", "TIMER", "TOUCHPAD", "ULP"][machine.wake_reason()]+"_WAKE"

print("Startup secs=%.3f: %s, %s" % (timestartup, resetcause, wakereason))
rtcmemory = rtc.memory()
print("rtcmemory:", rtcmemory, time.ticks_ms())

import uasyncio as asyncio
from machine import Pin, PWM
fconfig = dict(x.split()  for x in open("config.txt"))

pinyellow = Pin(int(fconfig["pinyellow"]), Pin.OUT)
pingreen = PWM(Pin(int(fconfig["pingreen"])), duty=0)
pinred = PWM(Pin(int(fconfig["pinred"])), duty=0)
pinrocker = Pin(int(fconfig["pinrocker"]), Pin.IN, Pin.PULL_DOWN)

rockerpwmfreq = const(60)
rockerflashfreq = const(4)
rockerfduration_toflash = const(4000)
rockerfduration_tosleep = const(5000)
connectbylimit = const(10000)

rockervalueprev = 0  # if left as -1 then no change has happened
rockervalue = -1
rockertime = 0
rockerfduration = 0

async def ledrockertask():
    global rockervalue, rockervalueprev, rockertime, rockerfduration
    while True:
        rockervalueprev, rockervalue = rockervalue, pinrocker.value()
        if rockervalue != rockervalueprev:
            pingreen.freq(rockerpwmfreq)
            rockertime = time.ticks_ms()
            rockerfduration = 0
            if rockervalueprev == -1 and machine.wake_reason() == machine.TIMER_WAKE:
                rockerfduration = rockerfduration_toflash
            if rockervalue == 1:
                pingreen.duty(20)
                pinred.duty(0)
            else:
                pinred.duty(20)
                pingreen.duty(0)
        else:
            rockerfduration = time.ticks_ms() - rockertime
            if rockerfduration > rockerfduration_toflash and pingreen.freq() != rockerflashfreq:
                pingreen.freq(rockerflashfreq)
        await asyncio.sleep_ms(100)

async def resleeptask():
    pinyellow.value(1)
    await asyncio.sleep_ms(50)
    pinyellow.value(0)
    t0 = time.ticks_ms()
    while True:
        if rockerfduration >= rockerfduration_tosleep:
            pinyellow.value(1)
            dt = time.ticks_ms() - t0
            if client.isconnected() or dt > connectbylimit:
                break
            await asyncio.sleep_ms(50)
            pinyellow.value(0)
        await asyncio.sleep_ms(300)
        
    msg = "%d %s %.3f" % (rockervalue, rtcmemory.decode(), timestartup)
    if client.isconnected():
        await client.publish(boardname+b"/direction", msg)
    esp32.wake_on_ext0(pinrocker, (esp32.WAKEUP_ANY_HIGH if rockervalue == 0 else esp32.WAKEUP_ALL_LOW))
    timeatsleep = ftime()
    print("time at sleep:", timeatsleep)
    rtc.memory("%d %.3f %.3f %.3f %s %s" % (rockervalue, timestartup, timeconnected, timeatsleep, resetcause, wakereason))
    machine.deepsleep(int(fconfig["deepsleepsec"])*1000)
        

from mqtt_as import config, MQTTClient
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
client = MQTTClient(config)
boardname = fconfig["boardname"].encode()

async def mqtttask():
    global timeconnected
    try:
        await client.connect()
        timeconnected = ftime()
        print("time at connect:", timeconnected)
    except OSError as e:
        print("client connect error", [e])

aloop = asyncio.get_event_loop()
aloop.create_task(ledrockertask())
aloop.create_task(resleeptask())
aloop.create_task(mqtttask())
aloop.run_forever()


Sent 105 lines (3740 bytes) to main.py.


In [7]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [9]:
async def hi(t):
    await asyncio.sleep(t)
    print("go error", t)
    raise OSError()
    
aloop = asyncio.get_event_loop()
aloop.create_task(hi(2))
aloop.create_task(hi(3))
aloop.run_forever()


......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [3]:
# Below is code to test whether the timeouts work

In [37]:
%sendtofile main.py

import uasyncio as asyncio
fconfig = dict(x.split()  for x in open("config.txt"))

from mqtt_as import config, MQTTClient
import mqtt_as
mqtt_as.LOBO = True

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
client = MQTTClient(config)
boardname = fconfig["boardname"].encode()

async def mqtttask():
    print("enter connect")
    try:
        await client.connect()
    except OSError as e:
        print("client connect error", [e])

aloop = asyncio.get_event_loop()
aloop.run_until_complete(mqtttask())


Sent 23 lines (624 bytes) to main.py.
